In [1]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [1]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster

import requests
import json


#### Loading and understanding

In [10]:
file_ign_44 = '../Data/BDCARTO/44_Loire_Atlantique/1_DONNEES_LIVRAISON_2024-12-00080/data.gpkg'

### Contour de la Baule

In [11]:
# Charger un fichier GeoPackage
gdf_commune = gpd.read_file(file_ign_44, layer="commune")

# Le code postal a l'air d'être un identifiant unique
gdf_la_baule = gdf_commune[gdf_commune['code_postal'] == '44500'].to_crs('4326')
ign_geometry_la_baule = gdf_la_baule.iloc[0].geometry  # Extract the polygon geometry
gdf_la_baule

,cleabs,code_insee,code_insee_de_l_arrondissement,code_insee_de_la_collectivite_terr,code_insee_du_departement,code_insee_de_la_region,population,surface_en_ha,code_postal,nom_officiel,chef_lieu_d_arrondissement,chef_lieu_de_collectivite_terr,chef_lieu_de_departement,chef_lieu_de_region,capitale_d_etat,date_du_recensement,organisme_recenseur,lien_vers_chef_lieu,code_siren,geometry
266,COMMUNE_0000000009745827,44055,3,44D,44,52,16468,2219,44500,La Baule-Escoublac,False,False,False,False,False,2021-01-01,INSEE,PAIHABIT0000000028858387,214400558,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3..."


# Comparaison IRIS

In [19]:
gdf_iris_contours = gpd.read_file('../data/IRIS/contours-iris.gpkg')

gdf_iris_contours['code_insee'] = pd.to_numeric(gdf_iris_contours['code_insee'], errors='coerce')
gdf_iris_contours = gdf_iris_contours.dropna(subset=['code_insee'])
gdf_iris_contours['code_insee'] = gdf_iris_contours['code_insee'].astype(int)

# Keep dataframe for La Baule
gdf_baule_iris = gdf_iris_contours[gdf_iris_contours['code_insee'] ==  44055]
gdf_baule_iris = gdf_baule_iris.to_crs("EPSG:4326")

In [26]:
import pandas as pd
df_iris_hab = pd.read_excel("../data/IRIS/base-ic-logement-2021_xlsx/base-ic-logement-2021.xlsx", header=5)  # Load first sheet
df_iris_hab

,IRIS,REG,DEP,UU2020,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,TYP_IRIS,...,P21_RP_GARL,P21_RP_VOIT1P,P21_RP_VOIT1,P21_RP_VOIT2P,P21_RP_HABFOR,P21_RP_CASE,P21_RP_MIBOIS,P21_RP_MIDUR,C21_RP_HSTU1P,C21_RP_HSTU1P_SUROCC
0,010010000,84,01,01000,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),Z,...,295.928974,331.689275,109.305986,222.383289,NaN,NaN,NaN,NaN,347.030546,0.000000
1,010020000,84,01,01000,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),Z,...,88.020753,110.176596,48.567803,61.608793,NaN,NaN,NaN,NaN,114.880886,0.000000
2,010040101,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Pérouses-Triangle d'Activités,H,...,555.223980,664.163146,427.860413,236.302733,NaN,NaN,NaN,NaN,690.741933,15.369322
3,010040102,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,H,...,1230.295598,1471.279120,1007.413577,463.865543,NaN,NaN,NaN,NaN,1616.521346,84.819794
4,010040201,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Centre-Saint-Germain-Vareilles,H,...,1497.804738,1773.646798,1217.182461,556.464337,NaN,NaN,NaN,NaN,2252.500811,77.971419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49275,974240101,4,974,9D201,97424,Cilaos,ZZZZZZ,9742401,La Ville-Les Thermes,H,...,457.338423,448.489264,337.161358,111.327907,3.185377,44.344216,46.502500,610.562472,702.851408,21.602113
49276,974240102,4,974,9D201,97424,Cilaos,ZZZZZZ,9742401,La Ville-Mare Sèche,H,...,424.963946,439.458810,313.997326,125.461484,2.076765,42.605839,112.058525,503.598459,655.700672,21.962523
49277,974240103,4,974,9D201,97424,Cilaos,ZZZZZZ,9742401,Bras Sec,H,...,232.906440,217.498142,141.526739,75.971404,1.038383,11.656769,6.271518,279.331295,285.270717,16.060916
49278,974240104,4,974,9D201,97424,Cilaos,ZZZZZZ,9742401,Peter Both-Palmiste Rouge,H,...,214.305943,277.648940,186.713955,90.934985,2.076765,30.678886,17.776171,368.452816,417.973225,32.190211


In [28]:
gdf_baule_iris.head()

,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry
1851,IRIS____0000000440550101,44055,La Baule-Escoublac,0101,440550101,La Baule les Pins,H,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47..."
3039,IRIS____0000000440550102,44055,La Baule-Escoublac,0102,440550102,Centre-Benoît,H,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47..."
4924,IRIS____0000000440550103,44055,La Baule-Escoublac,0103,440550103,Gare-Grand Clos,H,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47...."
5788,IRIS____0000000440550104,44055,La Baule-Escoublac,0104,440550104,Beslon,H,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47..."
6709,IRIS____0000000440550105,44055,La Baule-Escoublac,0105,440550105,Escoublac,H,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47..."


In [34]:
cols = ['P21_MEN', 'P21_PMEN']
df_iris_hab[df_iris_hab['IRIS'] == str(440550101)][cols]

,P21_MEN,P21_PMEN
20661,1495.231665,2346.994576


In [27]:
df_iris_hab.head()

,IRIS,REG,DEP,UU2020,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,TYP_IRIS,...,P21_RP_GARL,P21_RP_VOIT1P,P21_RP_VOIT1,P21_RP_VOIT2P,P21_RP_HABFOR,P21_RP_CASE,P21_RP_MIBOIS,P21_RP_MIDUR,C21_RP_HSTU1P,C21_RP_HSTU1P_SUROCC
0,010010000,84,01,01000,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),Z,...,295.928974,331.689275,109.305986,222.383289,NaN,NaN,NaN,NaN,347.030546,0.000000
1,010020000,84,01,01000,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),Z,...,88.020753,110.176596,48.567803,61.608793,NaN,NaN,NaN,NaN,114.880886,0.000000
2,010040101,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Pérouses-Triangle d'Activités,H,...,555.223980,664.163146,427.860413,236.302733,NaN,NaN,NaN,NaN,690.741933,15.369322
3,010040102,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,H,...,1230.295598,1471.279120,1007.413577,463.865543,NaN,NaN,NaN,NaN,1616.521346,84.819794
4,010040201,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100402,Centre-Saint-Germain-Vareilles,H,...,1497.804738,1773.646798,1217.182461,556.464337,NaN,NaN,NaN,NaN,2252.500811,77.971419


In [72]:
# Ensure both columns are the same type (string is safer for joins)
gdf_baule_iris["code_iris"] = gdf_baule_iris["code_iris"].astype(str)
df_iris_hab["IRIS"] = df_iris_hab["IRIS"].astype(str)

# Perform the left join
gdf_merged = gdf_baule_iris.merge(df_iris_hab[["IRIS", "P21_MEN", "P21_PMEN"]], left_on="code_iris", right_on="IRIS", how="left")

# Drop redundant IRIS column if needed
gdf_merged.drop(columns=["IRIS"], inplace=True)

# Check result
gdf_merged.head()

,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry,P21_MEN,P21_PMEN
0,IRIS____0000000440550101,44055,La Baule-Escoublac,0101,440550101,La Baule les Pins,H,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",1495.231665,2346.994576
1,IRIS____0000000440550102,44055,La Baule-Escoublac,0102,440550102,Centre-Benoît,H,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",1115.739274,1616.919056
2,IRIS____0000000440550103,44055,La Baule-Escoublac,0103,440550103,Gare-Grand Clos,H,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",1260.653224,1869.032071
3,IRIS____0000000440550104,44055,La Baule-Escoublac,0104,440550104,Beslon,H,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",1257.854803,2408.488869
4,IRIS____0000000440550105,44055,La Baule-Escoublac,0105,440550105,Escoublac,H,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",1690.103667,3511.317978


In [73]:
nb_men = gdf_merged.P21_MEN.sum()
nb_hab = gdf_merged.P21_PMEN.sum()

### Calcul population par IRIS (Heatmap)

In [74]:
# Centrer la carte sur La Baule (approx.)
center_lat, center_lon = ign_geometry_la_baule.centroid.y, ign_geometry_la_baule.centroid.x
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    ign_geometry_la_baule,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Add the city boundary contour
folium.GeoJson(
    gdf_baule_iris,
    name="La Baule",
    style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.1},
    popup=folium.GeoJsonPopup(fields=['nom_iris'], labels=False)
).add_to(m)


# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

# Estimation de la demande liée aux habitants

In [75]:
# consommation d'un habitant qui roule en EV
avg_dist_hab = 30
conso_km = 0.2
avg_demand_hab = avg_dist_hab * conso_km

# Nb de EV dans la ville et qui chargent chez eux 
nb_voitures = 15267
ratio_voit_hab = nb_voitures / nb_hab
ratio_ev_baule = 0.046 # 4.6%
ratio_charge_exterieur = 0.6

# Coef pour passer du nb habitant d'une zone à la demande de la zone liée à sa pop
avg_pop_to_demand = ratio_voit_hab * ratio_ev_baule * ratio_charge_exterieur * avg_demand_hab

In [76]:
gdf_merged['avg_demand_kWh'] = gdf_merged['P21_PMEN'] * avg_pop_to_demand
gdf_merged

,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry,P21_MEN,P21_PMEN,avg_demand_kWh
0,IRIS____0000000440550101,44055,La Baule-Escoublac,0101,440550101,La Baule les Pins,H,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",1495.231665,2346.994576,365.826449
1,IRIS____0000000440550102,44055,La Baule-Escoublac,0102,440550102,Centre-Benoît,H,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",1115.739274,1616.919056,252.029452
2,IRIS____0000000440550103,44055,La Baule-Escoublac,0103,440550103,Gare-Grand Clos,H,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",1260.653224,1869.032071,291.326351
3,IRIS____0000000440550104,44055,La Baule-Escoublac,0104,440550104,Beslon,H,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",1257.854803,2408.488869,375.411575
4,IRIS____0000000440550105,44055,La Baule-Escoublac,0105,440550105,Escoublac,H,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",1690.103667,3511.317978,547.309738
5,IRIS____0000000440550106,44055,La Baule-Escoublac,0106,440550106,Le Guézy,H,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3...",2235.585399,4467.253898,696.311635


In [ ]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a colormap based on the number of people in each carreau
min_demand = gdf_merged['avg_demand_kWh'].min()  # Minimum number of people
max_demand = gdf_merged['avg_demand_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_demand, max_demand)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_merged.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['avg_demand_kWh']  # Assuming this column exists
    nom = row['nom_iris']  # Assuming this column exists

    # Create a popup displaying the number of people
    text = f"{nom} : Populatio demand={demand:.2f}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)
    
# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

Données:
* Nb de véhicules / habitant => OK
* Nb de EV / véhicule => OK
* Demande moyenne par habitant / jour

To-do pour chaque zone
* Faire la heatmap
* Récup le nb de EV = nb d'habitant * nb_vehicule / hab * nb_ev / vehicule
* Récup la demande par zone = nb de EV * demande moyenne / jour

In [78]:
gdf_merged.drop(columns=['P21_MEN', 'P21_PMEN'], inplace=True)
gdf_merged.to_file(f"../data/processed/demand_per_iris_hab.geojson", driver="GeoJSON")

# Adding both demands

In [90]:
df_traffic_demand = gpd.read_file("../data/RP2020_MOBPRO_csv/demand_per_iris_150km.geojson")
df_traffic_demand.drop(columns=['demand_km_ev', 'demand_km'], inplace=True)

In [99]:
df_traffic_demand["code_iris"] = df_traffic_demand["code_iris"].astype(str)
gdf_merged["code_iris"] = gdf_merged["code_iris"].astype(str)

# Perform the left join
iris_all_demands = gdf_merged.merge(df_traffic_demand[["code_iris", "demand_kwh_ev"]], on="code_iris", how="left")
iris_all_demands['total_demand_kwh'] = iris_all_demands['avg_demand_kWh'] + iris_all_demands['demand_kwh_ev']
iris_all_demands.head()

,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry,avg_demand_kWh,demand_kwh_ev,total_demand_kwh
0,IRIS____0000000440550101,44055,La Baule-Escoublac,0101,440550101,La Baule les Pins,H,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",365.826449,0.000000,365.826449
1,IRIS____0000000440550102,44055,La Baule-Escoublac,0102,440550102,Centre-Benoît,H,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",252.029452,0.000000,252.029452
2,IRIS____0000000440550103,44055,La Baule-Escoublac,0103,440550103,Gare-Grand Clos,H,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",291.326351,0.000000,291.326351
3,IRIS____0000000440550104,44055,La Baule-Escoublac,0104,440550104,Beslon,H,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",375.411575,0.000000,375.411575
4,IRIS____0000000440550105,44055,La Baule-Escoublac,0105,440550105,Escoublac,H,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",547.309738,80.974664,628.284402


In [100]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a colormap based on the number of people in each carreau
min_demand = iris_all_demands['total_demand_kwh'].min()  # Minimum number of people
max_demand = iris_all_demands['total_demand_kwh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_demand, max_demand)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in iris_all_demands.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['total_demand_kwh']  # Assuming this column exists
    nom = row['nom_iris']  # Assuming this column exists

    # Create a popup displaying the number of people
    text = f"{nom} : Total demand={demand:.2f}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)
    
# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m